In [1]:
from aws_tools.cloudwatch_logging import logger
from aws_tools.project_config import get_aws_config

import pandas as pd
import boto3

In [2]:
import commendaroo.data_transformer.model_data as mdt

from scipy import sparse
import implicit
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import json
import os
import boto3
import tensorflow_hub as hub

from aws_tools.cloudwatch_logging import logger
from aws_tools.project_config import get_aws_config

In [3]:
data_implicit = mdt.get_data()

2021-03-03 15:02:31,610 [INFO ]  Getting the data
2021-03-03 15:02:31,611 [INFO ]  Creating views tables
2021-03-03 15:03:56,893 [INFO ]  Creating purchases/rentals/PPVs tables
2021-03-03 15:04:30,452 [INFO ]  Loading views data
2021-03-03 15:04:30,452 [INFO ]  starting Athena query ...
2021-03-03 15:12:34,781 [INFO ]  loading d8ec39dd-384c-4ff0-8c74-273ccc90f59e.csv
2021-03-03 15:24:40,484 [INFO ]  Athena query complete: returning dataframe
2021-03-03 15:24:40,639 [INFO ]  Loading purchases/rentals/PPVs data
2021-03-03 15:24:40,640 [INFO ]  starting Athena query ...
2021-03-03 15:25:36,158 [INFO ]  loading 212b6a60-86c2-4391-a26c-aae0a3d1611d.csv
2021-03-03 15:25:50,769 [INFO ]  Athena query complete: returning dataframe
2021-03-03 15:26:09,175 [INFO ]  Data implicit loaded
2021-03-03 15:26:09,176 [INFO ]  Aggregate seasons into brands
2021-03-03 15:28:32,150 [INFO ]  Prepping data for implicit model
2021-03-03 15:29:10,121 [INFO ]  Getting synopsis


In [4]:
sts_client = boto3.client('sts')

assumed_role_object=sts_client.assume_role(
    RoleArn="arn:aws:iam::881289283440:role/pipeline/bttv-dar-prod-role-tvrecommend-prd-consrecommend",
    RoleSessionName="recommendAssume"
)

credentials = assumed_role_object['Credentials']

dynamo_client = boto3.client(
    'dynamodb',
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
    region_name='eu-west-1'
)

In [5]:
def get_full_table(TableName, dynamo_client):
    table = dynamo_client.scan(TableName = TableName)
    data = table['Items']
    while 'LastEvaluatedKey' in table:
        table = dynamo_client.scan(TableName = TableName, ExclusiveStartKey=table['LastEvaluatedKey'])
        data.extend(table['Items'])
    
    return data

In [6]:
table_b = get_full_table('tvrecommend-brand-prd', dynamo_client)
table_c = get_full_table('tvrecommend-collection-prd', dynamo_client)
table_p = get_full_table('tvrecommend-programme-prd', dynamo_client)

In [7]:
guids_unique = data_implicit['id_editorial'].drop_duplicates().tolist()

In [8]:
len(guids_unique)

17849

In [30]:
def get_metadata_table(guids_unique, table, type_production):
    output = []
    for i in table:
        try:
            print(i)
            if 'episodeNumber' not in list(i.keys()): # not episode or season AND not in data with history
                type_production = type_production
                guid = i['guid']['S']
                title = i['title']['S']
                synopsis = i['longDescription']['S']
                genre = i['genres']['L'][0]['S']
                subgenres = ' '.join([x['S'] for x in i['subGenres']['L']])
                scheduler_channel = i['schedulerChannels']['L'][0]['S']
                rating = i['rating']['S']
                #
                if (guid not in guids_unique):
                    history = 1
                else:
                    history = 0
                #
                output.append(
                    [guid,
                    type_production,
                    title,
                    synopsis,
                    genre,
                    subgenres,
                    scheduler_channel,
                    rating,
                    history]
                )
        except:
            pass
    data = pd.DataFrame(
        output,
        columns=[
            'id_editorial',
            'type_production',
            'title',
            'synopsis',
            'genre',
            'subgenres',
            'scheduler_channel',
            'rating',
            'history'
        ]
    )
    
    return data

In [15]:
def get_metadata_table(guids_unique, table, type_production):
    output = []
    for i in table:
        try:
            if 'episodeNumber' not in list(i.keys()): # not episode or season AND not in data with history
                type_production = type_production
                guid = i['guid']['S']
                title = i['title']['S']
                synopsis = i['longDescription']['S']
                genre = i['genres']['L'][0]['S']
                subgenres = ' '.join([x['S'] for x in i['subGenres']['L']])
                scheduler_channel = i['schedulerChannels']['L'][0]['S']
                try:
                    rating = i['rating']['S']
                except:
                    for j in table_p:
                        if j['guid']['S'] == i['anchors']['L'][0]['M']['programGuid']['S']:
                            rating = j['rating']['S']
        #
                if (guid not in guids_unique):
                    history = 1
                else:
                    history = 0
                #
                output.append(
                    [guid,
                    type_production,
                    title,
                    synopsis,
                    genre,
                    subgenres,
                    scheduler_channel,
                    rating,
                    history]
                )
        except:
            #print(i['guid']['S'])
            pass

    data_nohist = pd.DataFrame(
        output,
        columns=[
            'id_editorial',
            'type_production',
            'title',
            'synopsis',
            'genre',
            'subgenres',
            'scheduler_channel',
            'rating',
            'history'
        ]
    )
    return data_nohist

In [16]:
data_b = get_metadata_table(guids_unique, table_b, 'BRAND')
data_c = get_metadata_table(guids_unique, table_c, 'COLLECTION')
data_p = get_metadata_table(guids_unique, table_p, 'PROGRAM')

In [49]:
print(len(data_b))
print(len(data_c))
print(len(data_p))

print(sum(data_b['history']))
print(sum(data_c['history']))
print(sum(data_p['history']))

739
3669
8475
3
233
613


In [27]:
import commendaroo.data_transformer.get_metadata as gmd
import os
os.environ['ENVIRON_PROD'] = 'TESTING'

In [26]:
guids_unique = data_implicit['id_editorial'].drop_duplicates().tolist()

In [44]:
def return_description(table, ed_id, title):
    for d in table:
        if d['guid']['S'] == ed_id:
            try:
                return d['longDescription']['S']
            except:
                return title
    return title
    

def pull_synopsis(content, table_b, table_c, table_p):
    
    if content[1] == 'BRAND':
        return return_description(table_b, content[0], content[2])
        
    elif content[1] == 'COLLECTION':
        return return_description(table_c, content[0], content[2])
        
    elif content[1] == 'PROGRAM':
        return return_description(table_p, content[0], content[2])
        
    else:
        return ''
    
def get_full_table(TableName, dynamo_client):
    table = dynamo_client.scan(TableName = TableName)
    data = table['Items']
    while 'LastEvaluatedKey' in table:
        table = dynamo_client.scan(TableName = TableName, ExclusiveStartKey=table['LastEvaluatedKey'])
        data.extend(table['Items'])
    
    return data

def initiate_client():
    
    sts_client = boto3.client('sts')
    
    assumed_role_object=sts_client.assume_role(
        RoleArn="arn:aws:iam::881289283440:role/pipeline/bttv-dar-prod-role-tvrecommend-prd-consrecommend",
        RoleSessionName="recommendAssume"
    )
    
    credentials = assumed_role_object['Credentials']
    
    return boto3.client(
        'dynamodb',
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name='eu-west-1'
    )

def get_synopsis(data_implicit):
    
    dynamo_client = initiate_client()
    
    #response = dynamo_client.list_tables(Limit=10)
    
    table_b = get_full_table('tvrecommend-brand-prd', dynamo_client)
    table_c = get_full_table('tvrecommend-collection-prd', dynamo_client)
    table_p = get_full_table('tvrecommend-programme-prd', dynamo_client)
    
    
    data_unique = data_implicit[['id_editorial', 'type_production', 'title']].drop_duplicates()
    
    data_unique['synopsis'] = data_unique.apply(pull_synopsis, args=(table_b, table_c, table_p), axis = 1)
    
    data_implicit = data_implicit.merge(data_unique[['id_editorial', 'synopsis']], left_on='id_editorial', right_on='id_editorial', how='left') 
    
    return data_implicit

def get_metadata_table(guids_unique, table, type_production):
    output = []
    for i in table:
        try:
            if 'episodeNumber' not in list(i.keys()): # not episode or season AND not in data with history
                type_production = type_production
                guid = i['guid']['S']
                title = i['title']['S']
                synopsis = i['longDescription']['S']
                genre = i['genres']['L'][0]['S']
                subgenres = ' '.join([x['S'] for x in i['subGenres']['L']])
                scheduler_channel = i['schedulerChannels']['L'][0]['S']
                try:
                    rating = i['rating']['S']
                except:
                    for j in table_p:
                        if j['guid']['S'] == i['anchors']['L'][0]['M']['programGuid']['S']:
                            rating = j['rating']['S']
        #
                if (guid not in guids_unique):
                    history = 1
                else:
                    history = 0
                #
                output.append(
                    [guid,
                    type_production,
                    title,
                    synopsis,
                    genre,
                    subgenres,
                    scheduler_channel,
                    rating,
                    history]
                )
        except:
            #print(i['guid']['S'])
            pass

    data = pd.DataFrame(
        output,
        columns=[
            'id_editorial',
            'type_production',
            'title',
            'synopsis',
            'genre',
            'subgenres',
            'scheduler_channel',
            'rating',
            'history'
        ]
    )
    
    return data

In [37]:
dynamo_client = initiate_client()

In [39]:
table_b = get_full_table('tvrecommend-brand-prd', dynamo_client)
table_c = get_full_table('tvrecommend-collection-prd', dynamo_client)
table_p = get_full_table('tvrecommend-programme-prd', dynamo_client)

In [45]:
data_b = get_metadata_table(guids_unique, table_b, 'BRAND')

In [ ]:
data_c = get_metadata_table(guids_unique, table_c, 'COLLECTION')
data_p = get_metadata_table(guids_unique, table_p, 'PROGRAM')

In [53]:
print(len(data_b))
print(len(data_c))
print(len(data_p))

print(sum(data_b['history']))
print(sum(data_c['history']))
print(sum(data_p['history']))

739
3669
8475
3
233
613


In [50]:
data_nohist = pd.concat([data_b, data_c, data_p], ignore_index=True)

In [52]:
data_nohist.head(2)

,id_editorial,type_production,title,synopsis,genre,subgenres,scheduler_channel,rating,history
0,movida_18040,BRAND,Crashing,Pete Holmes created and stars in this comedy s...,Comedy,Comedy Drama US,TV,15,0
1,movida_48227,BRAND,Weediquette,Krishna Andavolu investigates the radical tran...,Documentary,Factual,TV Replay,15,0


In [54]:
def rating_toNumeric(x):
    '''
    Turn duplicate the rating column in a form that is numeric so that they can be compared
    '''
    if x == 'u':
        return 0
    elif x == 'pg':
        return 1
    elif x == '12':
        return 2
    elif x == '15':
        return 3
    elif x == '18':
        return 4
    else:
        return 5

In [55]:
data_nohist['rating_n'] = data_nohist['rating'].apply(rating_toNumeric)

In [56]:
data_nohist.head(2)

,id_editorial,type_production,title,synopsis,genre,subgenres,scheduler_channel,rating,history,rating_n
0,movida_18040,BRAND,Crashing,Pete Holmes created and stars in this comedy s...,Comedy,Comedy Drama US,TV,15,0,3
1,movida_48227,BRAND,Weediquette,Krishna Andavolu investigates the radical tran...,Documentary,Factual,TV Replay,15,0,3


In [61]:
def create_model(data_implicit):
    '''
    '''
    
    logger.info('Training the model...')
    
    sparse_content_person = sparse.csr_matrix(
        (data_implicit['eventStrength'].astype(float), (data_implicit['id_editorial_simple'], data_implicit['id_user_simple']))
    )
    
    model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.1, iterations=50, use_gpu = False)
    alpha = 15
    data_tofit = (sparse_content_person * alpha).astype('double')
    model.fit(data_tofit)
    
    return model

def downloadDirectoryFroms3(bucket, remoteDirectoryName, project_folder):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket) 
    for object in bucket.objects.filter(Prefix = remoteDirectoryName):
        if not os.path.exists(os.path.dirname(object.key.split(project_folder)[-1])):
            os.makedirs(os.path.dirname(object.key.split(project_folder)[-1]))
        if object.key.split(project_folder)[-1][-1] != '/': #avoids trying to copy files from an empty folder
            bucket.download_file(object.key,object.key.split(project_folder)[-1])

def load_use():
    logger.info('Loading Universal Sentence Encoder')
    
    bucket = 'bt-data-science-playground'
    project_folder = 'nps-score-verbatim-text-analysis/'
    model_filename = 'model_objects_UniversalSentenceEncoders/4'
    
    # Download the relevant model objects
    downloadDirectoryFroms3(bucket, project_folder+model_filename, project_folder)
    
    return hub.load(model_filename)


def more_like_this_nohist(content_id, syn_vecs, syn_norms, othermd_vecs, othermd_norms, id_editorial_list_nohist, id_editorial_simple_list_nohist, type_production_list_nohist, rating_nohist, genre_nohist, scheduler_channel_nohist):
    
    ## synopsis
    scores_syn = syn_vecs.dot(syn_vecs[content_id,:])  / (syn_norms * syn_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    scores_syn =  MinMaxScaler().fit_transform(scores_syn.reshape(-1,1))[:,0]
    
    ## other metadata
    scores_othermd = othermd_vecs.dot(othermd_vecs[content_id,:])  / (othermd_norms * othermd_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    scores_othermd =  MinMaxScaler().fit_transform(scores_syn.reshape(-1,1))[:,0]
    
    ## sum scores
    a_syn = get_aws_config('model_parameters.json')['MLT_strength_synopsis']
    a_othermd = get_aws_config('model_parameters.json')['MLT_strength_othermetadata']
    #
    scores = ((a_syn * scores_syn) + (a_othermd * scores_othermd)) / (a_syn + a_othermd)
    
    # make zero the content not matching the same scheduler_channel
    if scheduler_channel_nohist[content_id] in ['Kids', 'Film', 'TV', 'TV Replay', 'Sport', 'Music']:
        scheduler_channel_logical = scheduler_channel_nohist == scheduler_channel_nohist[content_id]
        scheduler_channel_logical = np.array(scheduler_channel_logical.astype(int))
        #
        scores = scores * scheduler_channel_logical
    
    # make zero the content not matching the same genre
    genre_logical = genre_nohist == genre_nohist[content_id]
    genre_logical = np.array(genre_logical.astype(int))
    #
    scores = scores * genre_logical
    
    # make zero the content with 2 or more degrees higher rating except for kids (same rating only)
    if scheduler_channel_nohist[content_id] == 'Kids':
        rating_logical = rating_nohist == rating_nohist[content_id]
        rating_logical = np.array(rating_logical.astype(int))
        #
        scores = scores * rating_logical
    else:
        rating_logical = rating_nohist <= (rating_nohist[content_id] + 1)
        rating_logical = np.array(rating_logical.astype(int))
        #
        scores = scores * rating_logical
    
    similar = sorted(zip(id_editorial_list_nohist, scores[id_editorial_simple_list_nohist], type_production_list_nohist), key=lambda x: -x[1])
    
    return similar[:21] # return 21, so later can drop itself and will have at least 20

def more_like_this(content_id, content_vecs, content_norms, syn_vecs, syn_norms, othermd_vecs, othermd_norms, availability, id_editorial_list, id_editorial_simple_list, type_production_list, rating, genre, scheduler_channel):
    
    ## historical data
    scores_cont = content_vecs.dot(content_vecs[content_id,:])  / (content_norms * content_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    scores_cont =  MinMaxScaler().fit_transform(scores_cont.reshape(-1,1))[:,0]
    
    ## synopsis
    scores_syn = syn_vecs.dot(syn_vecs[content_id,:])  / (syn_norms * syn_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    scores_syn =  MinMaxScaler().fit_transform(scores_syn.reshape(-1,1))[:,0]
    
    ## other metadata
    scores_othermd = othermd_vecs.dot(othermd_vecs[content_id,:])  / (othermd_norms * othermd_vecs[content_id,:].sum()) # i.e. calculating cosine similarity, (A.B) / (|A| x |B|) --> |B| just a constant so effectively won't need it
    scores_othermd =  MinMaxScaler().fit_transform(scores_syn.reshape(-1,1))[:,0]
    
    ## sum scores
    a_cont = get_aws_config('model_parameters.json')['MLT_strength_historical']
    a_syn = get_aws_config('model_parameters.json')['MLT_strength_synopsis']
    a_othermd = get_aws_config('model_parameters.json')['MLT_strength_othermetadata']
    #
    scores = ((a_cont * scores_cont) + (a_syn * scores_syn) + (a_othermd * scores_othermd)) / (a_cont + a_syn + a_othermd)
    
    # make zero the content no longer avilable
    scores = scores * availability
    
    # make zero the content not matching the same scheduler_channel
    if scheduler_channel[content_id] in ['Kids', 'Film', 'TV', 'TV Replay', 'Sport', 'Music']:
        scheduler_channel_logical = scheduler_channel == scheduler_channel[content_id]
        scheduler_channel_logical = np.array(scheduler_channel_logical.astype(int))
        #
        scores = scores * scheduler_channel_logical
    
    # make zero the content not matching the same genre
    genre_logical = genre == genre[content_id]
    genre_logical = np.array(genre_logical.astype(int))
    #
    scores = scores * genre_logical
    
    # make zero the content with 2 or more degrees higher rating except for kids (same rating only)
    if scheduler_channel[content_id] == 'Kids':
        rating_logical = rating == rating[content_id]
        rating_logical = np.array(rating_logical.astype(int))
        #
        scores = scores * rating_logical
    else:
        rating_logical = rating <= (rating[content_id] + 1)
        rating_logical = np.array(rating_logical.astype(int))
        #
        scores = scores * rating_logical
    
    similar = sorted(zip(id_editorial_list, scores[id_editorial_simple_list], type_production_list), key=lambda x: -x[1])
    
    return similar[:21] # return 21, so later can drop itself and will have at least 20
    
def for_you(person_id, sparse_person_content, content_vecs_T, person_vecs, availability, id_editorial_list, id_editorial_simple_list, type_production_list, id_user_list, id_user_simple_list):

    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()

    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1

    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0

    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs_T)

    rec_vector = MinMaxScaler().fit_transform(rec_vector.reshape(-1,1))[:,0]

    # Multiply by zero the scores of items already interacted with
    recommend_vector = person_interactions * rec_vector
    
    # make zero the content no longer avilable
    recommend_vector = recommend_vector * availability
    
    recs = sorted(zip(id_editorial_list, recommend_vector[id_editorial_simple_list], type_production_list), key=lambda x: -x[1])
    
    user_id = id_user_list[id_user_simple_list.index(person_id)]
    
    return recs[:50], user_id


def get_recommendations(data_implicit, model, data_nohist):
    '''
    '''
    
    logger.info('Creating recommendations...')
    
    # (sorting first alphabetically by status so ACTIVE versions of same items appears first over INACTIVE versions)
    id_editorial_legend = data_implicit[['id_editorial', 'id_editorial_simple', 'type_production', 'end_date', 'start_date', 'status', 'rating_n', 'genre', 'scheduler_channel', 'synopsis', 'sub_genres']].sort_values(by=['status','sub_genres']).drop_duplicates(subset=['id_editorial', 'id_editorial_simple']).sort_values(by='id_editorial_simple')

    id_editorial_simple_list = id_editorial_legend['id_editorial_simple'].tolist()
    id_editorial_list = id_editorial_legend['id_editorial'].tolist()
    type_production_list = id_editorial_legend['type_production'].tolist()

    # get array with 1 or 0 based on availability of content today + status
    availability = (id_editorial_legend['status'] == 'ACTIVE') & (id_editorial_legend['end_date'] > datetime.today().strftime('%Y-%m-%d')) & (id_editorial_legend['start_date'] <= datetime.today().strftime('%Y-%m-%d'))
    availability = np.array(availability.astype(int))
    
    scheduler_channel = np.array(id_editorial_legend['scheduler_channel'].tolist())
    
    genre = np.array(id_editorial_legend['genre'].tolist())
    
    sub_genres = [str(x).lower().replace(' ', '').replace(',', ' ') for x in id_editorial_legend['sub_genres'].tolist()]
    
    rating = np.array(id_editorial_legend['rating_n'].tolist())
    
    synopsis = id_editorial_legend['synopsis'].tolist()


    id_user_legend = data_implicit[['id_user', 'id_user_simple']].drop_duplicates(subset=['id_user', 'id_user_simple']).sort_values(by='id_user_simple')
    id_user_simple_list = id_user_legend['id_user_simple'].tolist()
    id_user_list = id_user_legend['id_user'].tolist()

    date_today = datetime.today().strftime('%Y-%m-%d')
    code_version = get_aws_config('model_parameters.json')['code_version']
    
    data_update_date =  {
        'view' : str(data_implicit[data_implicit['type_entitlement'] == 'VIEW']['event_date'].max()),
        'purchase' : str(data_implicit[data_implicit['type_entitlement'] == 'EVOD']['event_date'].max()),
        'rental' : str(data_implicit[data_implicit['type_entitlement'] == 'TVOD']['event_date'].max())
    }
    
    sparse_content_person = sparse.csr_matrix(
        (data_implicit['eventStrength'].astype(float), (data_implicit['id_editorial_simple'], data_implicit['id_user_simple']))
    )
    sparse_person_content = sparse.csr_matrix(
        (data_implicit['eventStrength'].astype(float), (data_implicit['id_user_simple'], data_implicit['id_editorial_simple']))
    )
    
    ### More Like This   
    logger.info('More Like This')
    
    content_vecs = model.item_factors
    content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|
    
    embed = load_use()
    syn_vecs = np.array(embed(synopsis))
    syn_norms = np.sqrt((syn_vecs * syn_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|
    
    vectorizer = CountVectorizer()
    othermd_vecs = vectorizer.fit_transform(sub_genres).toarray()
    othermd_norms = np.sqrt((othermd_vecs * othermd_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|

    # create the output for dynamoDB table
    output = []

    for index, i in enumerate(id_editorial_simple_list[:10]): 

        if availability[index] == 1: # check if content is available otherwise those recommendations will be wrong due to having multiplied scores by zero for unavailable content

            con_id = id_editorial_list[index]
            recs = more_like_this(i, content_vecs, content_norms, syn_vecs, syn_norms, othermd_vecs, othermd_norms, availability, id_editorial_list, id_editorial_simple_list, type_production_list, rating, genre, scheduler_channel)

            rec_guids = []
            for r in recs:
                
                if r[0] != con_id: # skip itself

                    rec_guids.append({
                        'guid' : r[0],
                        'score' : round(float(r[1]), 5),
                        'type' : r[2]
                    })

            output_rec = {'score_date' : date_today, 'data_update_date' : data_update_date, 'code_version' : code_version, 'recommendations' : rec_guids[:20]}

            #output.append([con_id, output_rec])
            
            #also append same recommendations with <type>|<guid> lookup
            con_id_type = type_production_list[index]
            output.append([con_id_type + '|' + con_id, output_rec])

    output_morelikethis = pd.DataFrame(output,columns=['content','recommendations'])

    output_morelikethis['recommendations'] = output_morelikethis['recommendations'].apply(lambda x: json.dumps(x)) # if already a dict

#     # remove any duplicates; these can still occur if some content has exactly the same watch history as other and gets most similar item itself and other stuff too
#     output_morelikethis = output_morelikethis.drop_duplicates(subset=['content']).reset_index(drop=True)

    logger.info('More Like This with history: done - found {} items'.format(len(output_morelikethis)))
    
    ####### More Like This for items without history 
    logger.info('More Like This for items without history')
    
    synopsis_nohist = data_nohist['synopsis']
    sub_genres_nohist = data_nohist['subgenres']
    history = data_nohist['history'] # this here is 1 if missingbhistory and needs MLT or 0 if already has history and can be skipped
    
    id_editorial_list_nohist = data_nohist['id_editorial'].tolist()
    id_editorial_simple_list_nohist = data_nohist['id_editorial'].astype("category").cat.codes.tolist()
    type_production_list_nohist = data_nohist['type_production'].tolist()
    rating_nohist = data_nohist['rating_n']
    genre_nohist = data_nohist['genre']
    scheduler_channel_nohist = data_nohist['scheduler_channel']
    
    syn_vecs = np.array(embed(synopsis_nohist))
    syn_norms = np.sqrt((syn_vecs * syn_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|
    
    vectorizer = CountVectorizer()
    othermd_vecs = vectorizer.fit_transform(sub_genres_nohist).toarray()
    othermd_norms = np.sqrt((othermd_vecs * othermd_vecs).sum(axis=1)) # i.e. calculating abs. value of the vector of each item -->  |A|

    # create the output for dynamoDB table
    output = []

    for index, i in enumerate(range(len(data_nohist))): 

        if history[index] == 1: # check if content is available otherwise those recommendations will be wrong due to having multiplied scores by zero for unavailable content

            con_id = data_nohist['id_editorial'].tolist()[index]
            recs = more_like_this_nohist(i, syn_vecs, syn_norms, othermd_vecs, othermd_norms, id_editorial_list_nohist, id_editorial_simple_list_nohist, type_production_list_nohist, rating_nohist, genre_nohist, scheduler_channel_nohist)

            rec_guids = []
            for r in recs:
                
                if r[0] != con_id: # skip itself

                    rec_guids.append({
                        'guid' : r[0],
                        'score' : round(float(r[1]), 5),
                        'type' : r[2]
                    })

            output_rec = {'score_date' : date_today, 'data_update_date' : data_update_date, 'code_version' : code_version, 'recommendations' : rec_guids[:20]}

            #output.append([con_id, output_rec])
            
            #also append same recommendations with <type>|<guid> lookup
            con_id_type = type_production_list[index]
            output.append([con_id_type + '|' + con_id, output_rec])

    output_morelikethis_nohist = pd.DataFrame(output,columns=['content','recommendations'])

    output_morelikethis_nohist['recommendations'] = output_morelikethis_nohist['recommendations'].apply(lambda x: json.dumps(x)) # if already a dict
    
    logger.info('More Like This without history: done - found {} items'.format(len(output_morelikethis_nohist)))
    
    output_morelikethis_full = pd.concat([output_morelikethis, output_morelikethis_nohist], ignore_index=True)
    logger.info('More Like This full: done - found {} items'.format(len(output_morelikethis_full)))
    #######
    
    
    ### For You
    logger.info('For You')
    content_vecs_T = model.item_factors.T
    person_vecs = model.user_factors

    # create the output for dynamoDB table
    output = []
    
    # Do For You for all users loaded in, unless testing
    users_list_foryou = id_user_simple_list.copy()
    #
    if "ENVIRON_PROD" in os.environ:
        if os.environ['ENVIRON_PROD'] == 'TESTING':
            users_tech_trial = [
                'V3006126692',
                'V3007158774',
                'V3009295704',
                'V3000606445',
                'V2283739102',
                'V2200004307',
                'V3009378451',
                'V3009436446',
                'V1000019727',
                'V3008138884',
                'V3003052251',
                'V3008613163',
                'V3578624855',
                'V3000143118',
                'V2200007285',
                'V3000163582',
                'V3000537931'
            ]
            # turn the VSID of the tech trail list to simple user ids
            users_list_foryou = [int(data_implicit[data_implicit['id_user'] == x]['id_user_simple'].iloc[0]) for x in users_tech_trial if x in id_user_list]
        
        
    for i in users_list_foryou: 

        recs, user_id = for_you(i, sparse_person_content, content_vecs_T, person_vecs, availability, id_editorial_list, id_editorial_simple_list, type_production_list, id_user_list, id_user_simple_list)

        rec_guids = []
        for r in recs:

            rec_guids.append({
                'guid' : r[0],
                'score' : round(float(r[1]), 5),
                'type' : r[2]
            })

        output_rec = {'score_date' : date_today, 'data_update_date' : data_update_date, 'code_version' : code_version, 'recommendations' : rec_guids}

        output.append([user_id, output_rec])
        
        if int(i+1) % 50_000 == 0:
            logger.info('For You calculated for {} users'.format(int(i)+1))
    logger.info('For You calculated for all users ({}).'.format(int(i)+1)) 
    
    ##### add 'anonymous' entry for most popular content
    logger.info('Adding anonymous in For You table')
    n_days = 9
    popular = data_implicit[(data_implicit['event_date'] + pd.DateOffset(n_days) >= date_today)]['id_editorial_simple'].value_counts()
    
    rec_guids = []
    scores_fake = sorted(list(range(len(popular.index.tolist()[:20]))), reverse=True)
    for count, i in enumerate(popular.index.tolist()[:20]):
        
        rec_guids.append({
                'guid' : data_implicit[data_implicit['id_editorial_simple'] == i]['id_editorial'].iloc[0],
                'score' : scores_fake[count],
                'type' : data_implicit[data_implicit['id_editorial_simple'] == i]['type_production'].iloc[0]
            })
        
    output_rec = {'score_date' : date_today, 'data_update_date' : data_update_date, 'code_version' : code_version, 'recommendations' : rec_guids}

    output.append(['anonymous', output_rec])
    #####

    output_foryou = pd.DataFrame(output,columns=['user','recommendations'])

    output_foryou['recommendations'] = output_foryou['recommendations'].apply(lambda x: json.dumps(x)) # if already a dict

    logger.info('For You: done')
    
    return output_foryou, output_morelikethis_full

In [34]:
model = create_model(data_implicit)

2021-03-03 15:54:36,474 [INFO ]  Training the model...
2021-03-03 15:54:48,363 [WARNI]  OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading


In [62]:
np.seterr(divide='ignore', invalid='ignore') # to avoid divide by nan warning (genre in music is nan)

recs_foryou, recs_morelikethis = get_recommendations(data_implicit, model, data_nohist)

2021-03-03 16:11:44,732 [INFO ]  Creating recommendations...
2021-03-03 16:13:34,222 [INFO ]  More Like This
2021-03-03 16:13:34,224 [INFO ]  Loading Universal Sentence Encoder
2021-03-03 16:14:12,332 [WARNI]  7 out of the last 9 calls to <function recreate_function.<locals>.restored_function_body at 0x7f0eb8b8ea60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2021-03-03 16:14:14,834 [WARNI]  8 out of the last 10 calls to <function recr

In [63]:
recs_morelikethis.head()

,content,recommendations
0,PROGRAM|movida_53508,"{""score_date"": ""2021-03-03"", ""data_update_date..."
1,PROGRAM|movida_28246,"{""score_date"": ""2021-03-03"", ""data_update_date..."
2,PROGRAM|movida_24188,"{""score_date"": ""2021-03-03"", ""data_update_date..."
3,PROGRAM|movida_10067275,"{""score_date"": ""2021-03-03"", ""data_update_date..."
4,PROGRAM|movida_10035671,"{""score_date"": ""2021-03-03"", ""data_update_date..."


In [64]:
len(recs_foryou)

16

In [63]:
for k in table_b:
    if k['guid']['S'] == 'movida_49007':
        print(k)

{'updatedAtSource': {'N': '1606830293000'}, 'backgroundImageUrl': {'S': 'content_providers/images/49007_f112c765eb583882_HRB.jpg'}, 'created': {'N': '1606830288675'}, 'sortTitle': {'S': 'Mountain Gorilla'}, 'anchors': {'L': []}, 'longDescription': {'S': "The story of the world's last mountain gorillas, protected by a dedicated band of humans who watch and care for them."}, 'straplineText': {'S': 'Eden'}, 'contentProviderIDList': {'L': []}, 'promoType': {'S': ''}, 'subGenres': {'L': []}, 'offerTypes': {'L': []}, 'updated': {'N': '1611545122301'}, 'schedulerChannels': {'L': []}, 'shortDescription': {'S': "The story of the world's last mountain gorillas, protected by a dedicated band of humans."}, 'guid': {'S': 'movida_49007'}, 'channels': {'L': []}, 'seasons': {'L': [{'M': {'guid': {'S': 'movida_10067653'}, 'offerTypes': {'L': []}}}]}, 'packshotImageUrl': {'S': 'content_providers/images/49007_a1924ca1d16169dc_HRP.jpg'}, 'displayTitle': {'S': 'Mountain Gorilla'}, 'genres': {'L': []}, 'car

In [59]:
len(table_b)

1069

In [58]:
data_nohist['history'].value_counts()

0    736
1      3
Name: history, dtype: int64

In [64]:
data_nohist.head()

,id_editorial,type_production,title,synopsis,genre,subgenres,scheduler_channel,rating,history
0,movida_18040,BRAND,Crashing,Pete Holmes created and stars in this comedy s...,Comedy,Comedy Drama US,TV,15,0
1,movida_48227,BRAND,Weediquette,Krishna Andavolu investigates the radical tran...,Documentary,Factual,TV Replay,15,0
2,movida_14118,BRAND,Critical,"With sixty minutes to save a life, the clock i...",Drama,Drama UK,TV,15,0
3,movida_56497,BRAND,ThunderCats Roar!,"After their home world is destroyed, the Thund...",6 Years +,6 Years + 10's Adventure Animation Fantasy Sup...,Kids,U,0
4,movida_50625,BRAND,Lost Treasures Of Egypt,"An immersive, action-packed and discovery-led ...",Documentary,Factual,TV Replay,15,0


In [41]:
i = table_b[0]

In [42]:
type_production = 'BRAND'
guid = i['guid']['S']
title = i['title']['S']
synopsis = i['longDescription']['S']
genre = i['genres']['L'][0]['S']
subgenres = ' '.join([x['S'] for x in i['subGenres']['L']])
scheduler_channel = i['schedulerChannels']['L'][0]['S']
rating = i['rating']['S']
if (guid not in guids_unique):
    history = 1
else:
    history = 0

KeyError: 'rating'

In [45]:
i.keys()

dict_keys(['updatedAtSource', 'offerStartDate', 'availableStartDate', 'availableEndDate', 'pricingStartDate', 'anchors', 'pricingEndDate', 'longDescription', 'contentProviderIDList', 'offerTypes', 'shortDescription', 'packshotImageUrl', 'displayTitle', 'carouselTags', 'contentGroups', 'scodes', 'backgroundImageUrl', 'created', 'subGenres', 'updated', 'schedulerChannels', 'guid', 'year', 'offerEndDate', 'channels', 'seasons', 'genres', 'title'])

In [53]:
i['anchors']['L'][0]['M']['programGuid']['S']

'BBJ1331795A'

In [40]:
output

[]

In [31]:
data_nohist_b = get_metadata_table(guids_unique, table_b)

{'updatedAtSource': {'N': '1575289171000'}, 'offerStartDate': {'N': '1491951600000'}, 'availableStartDate': {'N': '1491951600000'}, 'availableEndDate': {'N': '1767225540000'}, 'pricingStartDate': {'N': '1491951600000'}, 'anchors': {'L': [{'M': {'seasonGuid': {'S': 'movida_10011328'}, 'programGuid': {'S': 'BBJ1331795A'}, 'seasonNumber': {'N': '1'}, 'episodeNumber': {'N': '1'}, 'anchorType': {'S': 'default'}}}, {'M': {'seasonGuid': {'S': 'movida_10011328'}, 'programGuid': {'S': 'BBJ1331795A'}, 'seasonNumber': {'N': '1'}, 'anchorId': {'S': 'XXH'}, 'episodeNumber': {'N': '1'}, 'anchorType': {'S': 'cpRelated'}}}]}, 'pricingEndDate': {'N': '1767225540000'}, 'longDescription': {'S': 'Pete Holmes created and stars in this comedy series about a stand-up comic on the New York City comedy scene'}, 'contentProviderIDList': {'L': []}, 'offerTypes': {'L': [{'S': 'EST'}]}, 'shortDescription': {'S': 'Comedy series about a stand-up comic on the New York City comedy scene'}, 'packshotImageUrl': {'S': 'c

In [32]:
data_nohist_b

,id_editorial,type_production,title,synopsis,genre,subgenres,scheduler_channel,rating,history


In [34]:
table_b[0]['schedulerChannels']

{'L': [{'S': 'TV'}]}

In [5]:
table_b[0].keys()

dict_keys(['updatedAtSource', 'offerStartDate', 'availableStartDate', 'availableEndDate', 'pricingStartDate', 'anchors', 'pricingEndDate', 'longDescription', 'contentProviderIDList', 'offerTypes', 'shortDescription', 'packshotImageUrl', 'displayTitle', 'carouselTags', 'contentGroups', 'scodes', 'backgroundImageUrl', 'created', 'subGenres', 'updated', 'schedulerChannels', 'guid', 'year', 'offerEndDate', 'channels', 'seasons', 'genres', 'title'])

In [38]:
table_p[0].keys()

dict_keys(['subtitlesEnabled', 'updatedAtSource', 'episodeNumber', 'offerStartDate', 'availableStartDate', 'availableEndDate', 'adEnabled', 'credits', 'longDescription', 'productAssociations', 'offerTypes', 'shortDescription', 'contentProviderId', 'packshotImageUrl', 'displayTitle', 'carouselTags', 'contentGroups', 'scodes', 'backgroundImageUrl', 'rating', 'runtime', 'created', 'straplineText', 'subGenres', 'parentProgramGuidList', 'updated', 'schedulerChannels', 'parentInfo', 'guid', 'productGuids', 'year', 'offerEndDate', 'channels', 'genres', 'title'])

In [14]:
table_p[1].keys()

dict_keys(['subtitlesEnabled', 'updatedAtSource', 'offerStartDate', 'availableStartDate', 'availableEndDate', 'adEnabled', 'credits', 'pricingStartDate', 'sortTitle', 'pricingEndDate', 'longDescription', 'productAssociations', 'offerTypes', 'shortDescription', 'contentProviderId', 'packshotImageUrl', 'displayTitle', 'carouselTags', 'contentGroups', 'scodes', 'backgroundImageUrl', 'rating', 'runtime', 'created', 'subGenres', 'parentProgramGuidList', 'updated', 'schedulerChannels', 'parentInfo', 'guid', 'productGuids', 'year', 'offerEndDate', 'channels', 'genres', 'title'])

In [17]:
if 'episodeNumber' not in list(table_c[1].keys()):
    print('Not found')

Not found


In [ ]:
config_foryou = get_aws_config('for_you.json')
config_foryou['quoting'] = csv.QUOTE_NONE
config_foryou['quotechar'] = ""
config_foryou['escapechar'] = "\\"



data_foryou.columns = ['ID', 'prediction']
DynamoDBWriter(pandas_df=data_foryou, **config_foryou).run()